# Importing Essential Libraries

In [25]:
import mysql.connector
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

## Collect Minute Data From Yahoo Finance

In [26]:
def collect_minute_data(coin):
    data_frames = []
    end_date = datetime.now()
 
    for _ in range(5):
        start_date = end_date - timedelta(days=6)
        data = yf.download(coin, start=start_date, end=end_date, interval="1m")
        data_frames.append(data)
        end_date = start_date - timedelta(minutes=1)
    
    combined_data = pd.concat(data_frames)
    combined_data = combined_data[combined_data['Volume'] > 0]
    combined_data = combined_data.sort_values(by='Datetime', ascending=False)
    return combined_data
coin = "BUSD-USD"
df = collect_minute_data(coin)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


## Data Analysis 

In [27]:
df['date'] = df.index
df.reset_index(drop=True, inplace=True)
df = df[['date','Open','High','Low','Close','Volume']]
df.columns = ['date','open','high','low','close','volume']
df

,date,open,high,low,close,volume
0,2023-06-17 15:27:00+00:00,1.000050,1.000050,1.000050,1.000050,1763840
1,2023-06-17 15:22:00+00:00,1.000179,1.000179,1.000179,1.000179,10058752
2,2023-06-17 14:42:00+00:00,1.000045,1.000045,1.000045,1.000045,8960
3,2023-06-17 14:32:00+00:00,1.000081,1.000081,1.000081,1.000081,1070336
4,2023-06-17 14:27:00+00:00,1.000000,1.000000,1.000000,1.000000,1491200
...,...,...,...,...,...,...
5567,2023-05-18 22:27:00+00:00,0.999844,0.999844,0.999844,0.999844,163712
5568,2023-05-18 22:22:00+00:00,0.999909,0.999909,0.999909,0.999909,1603456
5569,2023-05-18 22:21:00+00:00,0.999918,0.999918,0.999918,0.999918,327168
5570,2023-05-18 22:16:00+00:00,0.999841,0.999841,0.999841,0.999841,4871680


## Data Ingestion to SQL Database

In [29]:
coin_name = coin.split('-')[0]
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123456789",
    database="cryptos"
)
mycursor = mydb.cursor()

create_table_query = f"""
CREATE TABLE {coin_name} (
    id INT AUTO_INCREMENT PRIMARY KEY,
    date timestamp,
    open FLOAT,
    high FLOAT,
    low FLOAT,
    close FLOAT,
    volume FLOAT
)
"""
mycursor.execute(create_table_query)
df_columns = df.columns
insert_query = f"INSERT INTO {coin_name} (date, open, high, low, close, volume) VALUES (%s, %s, %s, %s, %s, %s)"
values = df[df_columns].values.tolist()
mycursor.executemany(insert_query, values)
mydb.commit()
mycursor.close()
mydb.close()

## Fetching Data From SQL DataBase File

In [32]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123456789",
    database="cryptos"
)
mycursor = mydb.cursor()
query = "SELECT * FROM busd"
mycursor.execute(query)
result = mycursor.fetchall()
mydb.commit()
df = pd.DataFrame(result, columns=[desc[0] for desc in mycursor.description])
df

,id,date,open,high,low,close,volume
0,1,2023-06-17 15:27:00,1.000050,1.000050,1.000050,1.000050,1763840.0
1,2,2023-06-17 15:22:00,1.000180,1.000180,1.000180,1.000180,10058800.0
2,3,2023-06-17 14:42:00,1.000040,1.000040,1.000040,1.000040,8960.0
3,4,2023-06-17 14:32:00,1.000080,1.000080,1.000080,1.000080,1070340.0
4,5,2023-06-17 14:27:00,1.000000,1.000000,1.000000,1.000000,1491200.0
...,...,...,...,...,...,...,...
5567,5568,2023-05-18 22:27:00,0.999844,0.999844,0.999844,0.999844,163712.0
5568,5569,2023-05-18 22:22:00,0.999909,0.999909,0.999909,0.999909,1603460.0
5569,5570,2023-05-18 22:21:00,0.999918,0.999918,0.999918,0.999918,327168.0
5570,5571,2023-05-18 22:16:00,0.999841,0.999841,0.999841,0.999841,4871680.0
